In [ ]:
# Initialize NiveshakAI Query Engine
print("🚀 Initializing NiveshakAI Local RAG Engine...")

try:
    # Initialize the query engine with local models
    query_engine = QueryEngine('config/settings.yaml')
    print("✅ QueryEngine initialized successfully")
    
    # Test with a simple Philip Fisher question
    print("\n🤔 Testing with Philip Fisher knowledge...")
    
    test_query = "What are the key characteristics of a quality growth stock according to Philip Fisher?"
    print(f"Query: {test_query}")
    
    response = query_engine.process_query(test_query)
    
    print(f"\n📖 Knowledge Base Search: Found {len(response.sources)} relevant sources")
    print(f"🤖 AI Response Preview: {response.answer[:200]}...") 
    
except Exception as e:
    print(f"❌ Error initializing query engine: {e}")

# NiveshakAI Demo: Local AI-Powered Investment Analysis

**🚀 Welcome to NiveshakAI - Your Complete Local AI Investment Assistant**

This notebook demonstrates NiveshakAI's revolutionary local RAG (Retrieval-Augmented Generation) pipeline that runs entirely on your machine!

## 🎯 What Makes NiveshakAI Special:

### 🔒 **Completely Local & Private**
- **Local LLM**: DeepSeek R1 7B via Ollama (no API costs!)
- **Local Embeddings**: Nomic-embed-text model
- **Local Vector DB**: Qdrant for knowledge storage
- **Zero Data Sharing**: Everything stays on your machine

### 📚 **Pre-loaded Investment Knowledge**
- ✅ **Philip Fisher's "Common Stocks and Uncommon Profits"** already ingested (913 chunks)
- 🤖 Ask sophisticated questions about growth investing principles
- 📖 Add your own investment books for personalized analysis

### 🎯 **Three-Part Analysis Framework**
1. **Business & Qualitative Analysis** - Company operations, management, competitive position
2. **Financial Metrics & Judgement** - Key indicators with clear ✅ Good, ⚠️ Caution, ❌ Poor ratings
3. **Investment Recommendations** - AI-powered insights based on ingested knowledge

**🌟 Ready to explore? Let's dive into the local AI-powered investment analysis!**

---

## 🔧 Setup: Local RAG Infrastructure

Before we begin, let's verify that our local AI infrastructure is running:

1. **Ollama Service** - Local LLM and embedding models
2. **Qdrant Database** - Vector knowledge storage
3. **Philip Fisher Knowledge Base** - Pre-loaded investment wisdom

First, let's import libraries and initialize our local AI system.

In [ ]:
# Core imports
import sys
import os
import yaml
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Add the src directory to Python path
sys.path.append(os.path.join(os.getcwd(), 'src'))

# NiveshakAI Local RAG imports
from src.analysis.query import QueryEngine
from src.analysis.valuation import ValuationAnalyzer  
from src.ingestion.books import BookProcessor
from src.embedding.embedder import EmbeddingManager
from src.utils.logger import get_logger
from src.utils.financial_utils import FinancialCalculator

print("✅ NiveshakAI Local RAG modules imported successfully")
print(f"📁 Working directory: {os.getcwd()}")
print("🚀 Local AI Pipeline: Ollama + Qdrant + Philip Fisher Knowledge Base")

In [ ]:
# Test Local Infrastructure
import requests
import ollama

print("🔍 Testing Local AI Infrastructure...\n")

# Test Ollama connection
try:
    models = ollama.list()
    print(f"✅ Ollama connected - {len(models['models'])} models available")
    
    # Check for required models
    model_names = [m.get('name', 'Unknown') for m in models['models']]
    if 'deepseek-r1:7b' in model_names:
        print("   ✅ DeepSeek R1 7B model ready")
    if 'nomic-embed-text:latest' in model_names:
        print("   ✅ Nomic embedding model ready")
except Exception as e:
    print(f"❌ Ollama connection failed: {e}")

# Test Qdrant connection  
try:
    response = requests.get('http://localhost:6333')
    if response.status_code == 200:
        print("✅ Qdrant vector database connected")
except Exception as e:
    print(f"❌ Qdrant connection failed: {e}")

print("\n🎯 Local RAG infrastructure check complete!")

In [ ]:
# Load configuration
with open('config/settings.yaml', 'r') as f:
    settings = yaml.safe_load(f)
    
with open('config/persona.yaml', 'r') as f:
    persona = yaml.safe_load(f)

# Initialize logger
logger = get_logger('demo')

print("📋 Configuration loaded:")
print(f"   App: {settings['app']['name']} v{settings['app']['version']}")
print(f"   Investor: {persona['investor_profile']['name']}")
print(f"   Style: {persona['investor_profile']['philosophy']['style']}")
print(f"   Risk Tolerance: {persona['investor_profile']['risk_tolerance']}")

## 📚 Demo 1: Philip Fisher Knowledge Base

NiveshakAI comes pre-loaded with Philip Fisher's "Common Stocks and Uncommon Profits" - let's test the knowledge base with some investment questions!

## 📊 Define Analysis Structure

Let's create the structured analysis template that matches your preferred format.

# Load NiveshakAI configuration
with open('config/settings.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("📋 NiveshakAI Configuration:")
print(f"   🤖 LLM Provider: {config.get('llm', {}).get('provider', 'Unknown')}")
print(f"   🔗 Embedding Provider: {config.get('embedding', {}).get('provider', 'Unknown')}")
print(f"   💾 Vector DB: {config.get('vector_db', {}).get('provider', 'Unknown')}")
print(f"   📚 Vector Collection: {config.get('vector_db', {}).get('qdrant', {}).get('collection_name', 'Unknown')}")

# Load persona configuration 
with open('config/persona.yaml', 'r') as f:
    persona_config = yaml.safe_load(f)

print(f"\n👤 Investment Persona:")
print(f"   📊 Style: {persona_config.get('investor_profile', {}).get('philosophy', {}).get('style', 'Unknown')}")
print(f"   ⚖️ Risk Tolerance: {persona_config.get('investor_profile', {}).get('risk_tolerance', 'Unknown')}")
print(f"   📅 Horizon: {persona_config.get('investor_profile', {}).get('investment_horizon', 'Unknown')}")

print("\n✅ Configuration loaded successfully")

In [ ]:
class StockAnalysisTemplate:
    """Structured template matching the user's 3-part analysis format"""
    
    def __init__(self):
        self.business_analysis = {
            "what_does_company_do": "",
            "promoters_background": "",
            "manufacturing_details": "",
            "plant_capacity": "",
            "raw_materials": "",
            "clients_customers": "",
            "competitors": "",
            "major_shareholders": "",
            "regulatory_landscape": "",
            "product_launches": "",
            "geographical_expansion": "",
            "revenue_mix": "",
            "entry_barriers": "",
            "subsidiaries": ""
        }
        
        self.financial_metrics = {
            "gross_profit_margin": {"value": None, "judgement": ""},
            "revenue_growth": {"value": None, "judgement": ""},
            "eps_consistency": {"value": None, "judgement": ""},
            "debt_level": {"value": None, "judgement": ""},
            "inventory": {"value": None, "judgement": ""},
            "receivables_vs_sales": {"value": None, "judgement": ""},
            "cash_flow_operations": {"value": None, "judgement": ""},
            "roe": {"value": None, "judgement": ""},
            "business_diversity": {"value": None, "judgement": ""}
        }
        
        self.ratio_analysis = {
            "quick_ratio": {"value": None, "interpretation": ""},
            "current_ratio": {"value": None, "interpretation": ""},
            "pe_ratio": {"value": None, "interpretation": ""},
            "roce": {"value": None, "interpretation": ""},
            "roa": {"value": None, "interpretation": ""},
            "roe": {"value": None, "interpretation": ""},
            "dividend_yield": {"value": None, "interpretation": ""},
            "debt_equity": {"value": None, "interpretation": ""},
            "interest_coverage": {"value": None, "interpretation": ""},
            "price_to_book": {"value": None, "interpretation": ""},
            "price_to_sales": {"value": None, "interpretation": ""}
        }
    
    def get_judgement_emoji(self, value, good_threshold, poor_threshold, higher_is_better=True):
        """Return judgement emoji based on value thresholds"""
        if value is None:
            return "❓"
        
        if higher_is_better:
            if value >= good_threshold:
                return "✅"
            elif value >= poor_threshold:
                return "⚠️"
            else:
                return "❌"
        else:
            if value <= good_threshold:
                return "✅"
            elif value <= poor_threshold:
                return "⚠️"
            else:
                return "❌"

print("✅ Analysis template created")

## 🏢 Sample Company Analysis Data

Let's create sample data for demonstrating the analysis of different companies across various sectors.

In [ ]:
# Sample companies for analysis demonstration
sample_companies = {
    "RELIANCE": {
        "name": "Reliance Industries Limited",
        "sector": "Oil & Gas",
        "market_cap": 1500000,  # in crores
        "financials": {
            "revenue": 750000,
            "net_profit": 65000,
            "total_assets": 850000,
            "shareholders_equity": 650000,
            "current_assets": 180000,
            "current_liabilities": 120000,
            "debt": 280000,
            "cash_flow_operations": 75000,
            "eps": 97.5,
            "book_value": 975,
            "dividend_per_share": 9.0
        }
    },
    "TCS": {
        "name": "Tata Consultancy Services",
        "sector": "IT Services", 
        "market_cap": 1200000,
        "financials": {
            "revenue": 225000,
            "net_profit": 42000,
            "total_assets": 180000,
            "shareholders_equity": 140000,
            "current_assets": 120000,
            "current_liabilities": 65000,
            "debt": 5000,
            "cash_flow_operations": 45000,
            "eps": 115,
            "book_value": 380,
            "dividend_per_share": 75
        }
    },
    "HDFC_BANK": {
        "name": "HDFC Bank Limited",
        "sector": "Banking",
        "market_cap": 800000,
        "financials": {
            "revenue": 185000,
            "net_profit": 48000,
            "total_assets": 2100000,
            "shareholders_equity": 280000,
            "current_assets": 1800000,
            "current_liabilities": 1600000,
            "debt": 200000,
            "cash_flow_operations": 55000,
            "eps": 88,
            "book_value": 515,
            "dividend_per_share": 19.5
        }
    },
    "TRENT": {
        "name": "Trent Limited",
        "sector": "Retail",
        "market_cap": 85000,
        "financials": {
            "revenue": 6276,
            "net_profit": 876,
            "total_assets": 8500,
            "shareholders_equity": 3516,
            "current_assets": 2200,
            "current_liabilities": 1470,
            "debt": 682,
            "cash_flow_operations": 1365,
            "eps": 21.9,
            "book_value": 88,
            "dividend_per_share": 3.0
        }
    },
    "INFOSYS": {
        "name": "Infosys Limited", 
        "sector": "IT Services",
        "market_cap": 700000,
        "financials": {
            "revenue": 180000,
            "net_profit": 24500,
            "total_assets": 110000,
            "shareholders_equity": 85000,
            "current_assets": 75000,
            "current_liabilities": 32000,
            "debt": 8000,
            "cash_flow_operations": 26000,
            "eps": 58,
            "book_value": 200,
            "dividend_per_share": 38
        }
    }
}

print(f"✅ Sample data created for {len(sample_companies)} companies")
print("Companies:", ", ".join(sample_companies.keys()))

## 🧠 AI-Powered Analysis Functions

Now let's create functions that simulate how NiveshakAI would analyze companies using RAG and your analytical style.

In [ ]:
class NiveshakAIAnalyzer:
    """Main analyzer class that demonstrates NiveshakAI capabilities"""
    
    def __init__(self, persona_config):
        self.persona = persona_config
        self.template = StockAnalysisTemplate()
        
    def analyze_business_qualitative(self, company_symbol, company_data):
        """Simulate AI analysis of business and qualitative factors"""
        name = company_data['name']
        sector = company_data['sector']
        
        # This would normally use RAG to query annual reports and books
        business_analysis = {
            "what_does_company_do": f"{name} operates in the {sector} sector with strong market presence",
            "promoters_background": "Experienced management team with strong track record",
            "competitors": f"Major players in {sector} include established market leaders",
            "revenue_mix": f"Diversified revenue streams within {sector}",
            "entry_barriers": "High barriers due to capital requirements and regulatory compliance",
            "subsidiaries": "Strategic subsidiaries supporting core business"
        }
        
        return business_analysis
    
    def analyze_financial_metrics(self, company_symbol, financials):
        """Analyze financial metrics with judgements"""
        metrics = {}
        
        # Gross Profit Margin (assuming cost of sales is 60% of revenue)
        gpm = ((financials['revenue'] - financials['revenue'] * 0.6) / financials['revenue']) * 100
        gpm_judgement = "✅ Strong" if gpm > 30 else "⚠️ Moderate" if gpm > 15 else "❌ Weak"
        metrics['gross_profit_margin'] = {"value": f"{gpm:.1f}%", "judgement": gpm_judgement}
        
        # ROE
        roe = (financials['net_profit'] / financials['shareholders_equity']) * 100
        roe_judgement = "✅ Excellent" if roe > 20 else "⚠️ Good" if roe > 15 else "❌ Poor"
        metrics['roe'] = {"value": f"{roe:.1f}%", "judgement": roe_judgement}
        
        # Debt Level
        debt_ratio = financials['debt'] / financials['shareholders_equity']
        debt_judgement = "✅ Low" if debt_ratio < 0.3 else "⚠️ Moderate" if debt_ratio < 0.6 else "❌ High"
        metrics['debt_level'] = {"value": f"{debt_ratio:.2f}", "judgement": debt_judgement}
        
        # Cash Flow
        cf_margin = (financials['cash_flow_operations'] / financials['revenue']) * 100
        cf_judgement = "✅ Strong" if cf_margin > 15 else "⚠️ Adequate" if cf_margin > 8 else "❌ Weak"
        metrics['cash_flow_operations'] = {"value": f"{cf_margin:.1f}%", "judgement": cf_judgement}
        
        return metrics
    
    def analyze_ratios(self, company_symbol, financials, current_price=None):
        """Comprehensive ratio analysis"""
        ratios = {}
        
        # Current Ratio
        current_ratio = financials['current_assets'] / financials['current_liabilities']
        cr_interpretation = "✅ Healthy" if current_ratio > 1.5 else "⚠️ Adequate" if current_ratio > 1.0 else "❌ Poor"
        ratios['current_ratio'] = {"value": f"{current_ratio:.2f}", "interpretation": cr_interpretation}
        
        # Quick Ratio (assuming inventory is 30% of current assets)
        quick_assets = financials['current_assets'] * 0.7
        quick_ratio = quick_assets / financials['current_liabilities']
        qr_interpretation = "✅ Good" if quick_ratio > 1.0 else "⚠️ Adequate" if quick_ratio > 0.7 else "❌ Poor"
        ratios['quick_ratio'] = {"value": f"{quick_ratio:.2f}", "interpretation": qr_interpretation}
        
        # ROE
        roe = (financials['net_profit'] / financials['shareholders_equity']) * 100
        roe_interpretation = "✅ Excellent" if roe > 20 else "⚠️ Good" if roe > 15 else "❌ Poor"
        ratios['roe'] = {"value": f"{roe:.1f}%", "interpretation": roe_interpretation}
        
        # ROA
        roa = (financials['net_profit'] / financials['total_assets']) * 100
        roa_interpretation = "✅ Strong" if roa > 10 else "⚠️ Average" if roa > 5 else "❌ Weak"
        ratios['roa'] = {"value": f"{roa:.1f}%", "interpretation": roa_interpretation}
        
        # Debt to Equity
        de_ratio = financials['debt'] / financials['shareholders_equity']
        de_interpretation = "✅ Conservative" if de_ratio < 0.3 else "⚠️ Moderate" if de_ratio < 0.6 else "❌ High"
        ratios['debt_equity'] = {"value": f"{de_ratio:.2f}", "interpretation": de_interpretation}
        
        # Interest Coverage (estimated)
        interest_expense = financials['debt'] * 0.08  # Assuming 8% interest rate
        if interest_expense > 0:
            ebit = financials['net_profit'] + interest_expense  # Simplified EBIT
            interest_coverage = ebit / interest_expense
            ic_interpretation = "✅ Strong" if interest_coverage > 5 else "⚠️ Adequate" if interest_coverage > 2.5 else "❌ Risky"
            ratios['interest_coverage'] = {"value": f"{interest_coverage:.1f}x", "interpretation": ic_interpretation}
        
        # Dividend Yield (if current price provided)
        if current_price:
            div_yield = (financials['dividend_per_share'] / current_price) * 100
            dy_interpretation = "✅ Good" if div_yield > 2 else "⚠️ Low" if div_yield > 1 else "❌ Very Low"
            ratios['dividend_yield'] = {"value": f"{div_yield:.2f}%", "interpretation": dy_interpretation}
        
        return ratios
    
    def generate_investment_recommendation(self, company_symbol, analysis_results):
        """Generate investment recommendation based on analysis"""
        # Count positive, neutral, and negative indicators
        positive_count = 0
        neutral_count = 0
        negative_count = 0
        
        # Analyze financial metrics
        for metric in analysis_results['financial_metrics'].values():
            judgement = metric['judgement']
            if '✅' in judgement:
                positive_count += 1
            elif '⚠️' in judgement:
                neutral_count += 1
            elif '❌' in judgement:
                negative_count += 1
        
        # Analyze ratios
        for ratio in analysis_results['ratios'].values():
            interpretation = ratio['interpretation']
            if '✅' in interpretation:
                positive_count += 1
            elif '⚠️' in interpretation:
                neutral_count += 1
            elif '❌' in interpretation:
                negative_count += 1
        
        total_indicators = positive_count + neutral_count + negative_count
        positive_ratio = positive_count / total_indicators if total_indicators > 0 else 0
        
        # Generate recommendation
        if positive_ratio >= 0.7:
            recommendation = "🎯 STRONG BUY - Excellent fundamentals across multiple metrics"
            confidence = "High"
        elif positive_ratio >= 0.5:
            recommendation = "👍 BUY - Good fundamentals with some areas of caution"
            confidence = "Medium-High"
        elif positive_ratio >= 0.3:
            recommendation = "⚖️ HOLD - Mixed signals, monitor closely"
            confidence = "Medium"
        else:
            recommendation = "🚫 AVOID - Multiple red flags identified"
            confidence = "High"
        
        return {
            "recommendation": recommendation,
            "confidence": confidence,
            "positive_indicators": positive_count,
            "neutral_indicators": neutral_count,
            "negative_indicators": negative_count,
            "analysis_summary": f"Analysis shows {positive_count} positive, {neutral_count} neutral, and {negative_count} negative indicators"
        }

# Initialize the analyzer
analyzer = NiveshakAIAnalyzer(persona)
print("✅ NiveshakAI Analyzer initialized")

## 🎯 Live Analysis Demo

Let's analyze our sample companies using the NiveshakAI system and display results in your preferred format.

In [ ]:
def display_analysis(company_symbol, company_data, show_full=True):
    """Display complete analysis in the user's preferred format"""
    print(f"\n{'='*80}")
    print(f"📊 NIVESHAKAI ANALYSIS: {company_data['name']} ({company_symbol})")
    print(f"{'='*80}")
    
    # Perform analysis
    business_analysis = analyzer.analyze_business_qualitative(company_symbol, company_data)
    financial_metrics = analyzer.analyze_financial_metrics(company_symbol, company_data['financials'])
    ratios = analyzer.analyze_ratios(company_symbol, company_data['financials'])
    
    analysis_results = {
        'business_analysis': business_analysis,
        'financial_metrics': financial_metrics,
        'ratios': ratios
    }
    
    recommendation = analyzer.generate_investment_recommendation(company_symbol, analysis_results)
    
    if show_full:
        # Section 1: Business & Qualitative Analysis
        print("\n🏢 SECTION 1: BUSINESS & QUALITATIVE ANALYSIS")
        print("-" * 50)
        for key, value in business_analysis.items():
            formatted_key = key.replace('_', ' ').title()
            print(f"• {formatted_key}: {value}")
        
        # Section 2: Financial Metrics & Judgement
        print("\n💰 SECTION 2: FINANCIAL METRICS & JUDGEMENT")
        print("-" * 50)
        for metric, data in financial_metrics.items():
            formatted_metric = metric.replace('_', ' ').title()
            print(f"• {formatted_metric}: {data['value']} | {data['judgement']}")
        
        # Section 3: Ratio Analysis
        print("\n📈 SECTION 3: RATIO ANALYSIS")
        print("-" * 50)
        for ratio, data in ratios.items():
            formatted_ratio = ratio.replace('_', ' ').title()
            print(f"• {formatted_ratio}: {data['value']} | {data['interpretation']}")
    
    # Investment Recommendation
    print(f"\n🎯 INVESTMENT RECOMMENDATION")
    print("-" * 50)
    print(f"Decision: {recommendation['recommendation']}")
    print(f"Confidence: {recommendation['confidence']}")
    print(f"Summary: {recommendation['analysis_summary']}")
    
    return analysis_results, recommendation

print("✅ Analysis display function ready")

In [ ]:
# Analyze all sample companies
analysis_summary = []

for symbol, data in sample_companies.items():
    results, recommendation = display_analysis(symbol, data, show_full=False)
    
    analysis_summary.append({
        'Symbol': symbol,
        'Company': data['name'],
        'Sector': data['sector'],
        'Recommendation': recommendation['recommendation'],
        'Confidence': recommendation['confidence'],
        'Positive_Indicators': recommendation['positive_indicators'],
        'Neutral_Indicators': recommendation['neutral_indicators'], 
        'Negative_Indicators': recommendation['negative_indicators']
    })

print(f"\n✅ Analysis completed for {len(sample_companies)} companies")

## 📋 Portfolio Analysis Summary

Let's create a comparative summary of all analyzed companies.

In [ ]:
# Create summary DataFrame
summary_df = pd.DataFrame(analysis_summary)

# Display summary table
print("\n📊 PORTFOLIO ANALYSIS SUMMARY")
print("=" * 120)
print(summary_df.to_string(index=False))

# Create detailed comparison for top picks
print("\n\n🏆 TOP INVESTMENT PICKS (Based on Analysis)")
print("=" * 60)

# Sort by positive indicators
top_picks = summary_df.nlargest(3, 'Positive_Indicators')

for idx, row in top_picks.iterrows():
    print(f"\n{idx+1}. {row['Company']} ({row['Symbol']})")
    print(f"   Sector: {row['Sector']}")
    print(f"   {row['Recommendation']}")
    print(f"   Indicators: {row['Positive_Indicators']} positive, {row['Neutral_Indicators']} neutral, {row['Negative_Indicators']} negative")

## 🔍 Detailed Analysis Example

Let's see a complete detailed analysis for one company to demonstrate the full NiveshakAI capabilities.

In [ ]:
# Detailed analysis for TRENT (matching the ChatGPT conversation example)
print("\n🎯 DETAILED ANALYSIS EXAMPLE")
results, recommendation = display_analysis('TRENT', sample_companies['TRENT'], show_full=True)

## 📤 Export Analysis Results

NiveshakAI can export analysis results to various formats like Excel, CSV, and detailed reports.

In [ ]:
def export_to_excel(analysis_summary, filename="niveshak_analysis.xlsx"):
    """Export analysis to Excel format"""
    try:
        with pd.ExcelWriter(filename, engine='openpyxl') as writer:
            # Summary sheet
            summary_df = pd.DataFrame(analysis_summary)
            summary_df.to_excel(writer, sheet_name='Portfolio_Summary', index=False)
            
            # Individual company sheets
            for symbol, data in sample_companies.items():
                company_analysis, _ = display_analysis(symbol, data, show_full=False)
                
                # Create detailed DataFrame for each company
                company_df = pd.DataFrame({
                    'Metric': list(company_analysis['financial_metrics'].keys()) + list(company_analysis['ratios'].keys()),
                    'Value': [company_analysis['financial_metrics'][k]['value'] for k in company_analysis['financial_metrics'].keys()] + 
                            [company_analysis['ratios'][k]['value'] for k in company_analysis['ratios'].keys()],
                    'Assessment': [company_analysis['financial_metrics'][k]['judgement'] for k in company_analysis['financial_metrics'].keys()] + 
                                 [company_analysis['ratios'][k]['interpretation'] for k in company_analysis['ratios'].keys()]
                })
                
                company_df.to_excel(writer, sheet_name=symbol, index=False)
        
        print(f"✅ Analysis exported to {filename}")
        return filename
    except Exception as e:
        print(f"❌ Export failed: {str(e)}")
        return None

def export_to_csv(analysis_summary, filename="niveshak_summary.csv"):
    """Export summary to CSV format"""
    try:
        summary_df = pd.DataFrame(analysis_summary)
        summary_df.to_csv(filename, index=False)
        print(f"✅ Summary exported to {filename}")
        return filename
    except Exception as e:
        print(f"❌ CSV export failed: {str(e)}")
        return None

# Export results
excel_file = export_to_excel(analysis_summary)
csv_file = export_to_csv(analysis_summary)

print(f"\n📁 Files created in: {os.getcwd()}")
if excel_file:
    print(f"   📊 Excel: {excel_file}")
if csv_file:
    print(f"   📋 CSV: {csv_file}")

## 🔗 Integration with Real Data

In a real implementation, NiveshakAI would:

1. **Ingest Annual Reports**: Use `src/ingestion/reports.py` to process PDF annual reports
2. **Build Knowledge Base**: Extract financial data and store in vector database
3. **RAG-Based Analysis**: Query embedded finance books and reports for insights
4. **Persona-Based Filtering**: Apply your investment preferences and risk tolerance
5. **Live Data Integration**: Connect to financial APIs for real-time data

### Sample Integration Code:

```python
# Real implementation would look like:
from src.ingestion.reports import ReportProcessor
from src.embedding.embedder import EmbeddingManager

# Process annual report
processor = ReportProcessor()
report_data = processor.process_pdf('data/annual_reports/company_ar_2024.pdf')

# Create embeddings
embedding_manager = EmbeddingManager(settings)
embedding_manager.store_embeddings(report_data)

# Query for analysis
query_result = embedding_manager.query(
    "What is the company's revenue growth and profitability trend?"
)

# Generate structured analysis
ai_analyzer = AIAnalyzer(settings, persona)
analysis = ai_analyzer.analyze_company(query_result, company_symbol)
```

## 🚀 Next Steps

To use NiveshakAI with real data:

### 1. Configure API Keys
```bash
# Edit config/settings.yaml
api_keys:
  openai_api_key: "your-actual-openai-key"
  alpha_vantage_key: "your-alpha-vantage-key"
```

### 2. Set Up Vector Database
```bash
# Start local Qdrant instance
./scripts/run_local_vector_db.sh start
```

### 3. Ingest Finance Books
```bash
# Add finance books to data/books/
python3 -m src.cli.ingest_books --directory data/books/
```

### 4. Process Annual Reports
```bash
# Add annual reports to data/annual_reports/
python3 -m src.cli.ingest_reports --directory data/annual_reports/
```

### 5. Run Analysis
```bash
# Analyze a company
python3 -m src.cli.analyze --company RELIANCE --format structured
```

### 6. Customize Your Persona
Edit `config/persona.yaml` to match your:
- Investment philosophy
- Risk tolerance
- Sector preferences
- Financial criteria

---

**🤖 NiveshakAI Demo Complete!**

This notebook demonstrated the core capabilities of NiveshakAI using your preferred analysis format. The system can provide structured, professional investment analysis with clear judgments and actionable recommendations.